## Load relevant packages

In [ ]:
# Import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

## Load data files

In [ ]:
#================================================================================================
# USER INPUT!
# Here I specify which data files need reading in
datafilenames = ["train.csv","test.csv"]
#================================================================================================

alldataframes = [pd.read_csv(filename) for filename in datafilenames]

### Functions for detecting dirty data and cleaning it

In [ ]:
# This function takes a column and does a tally of the different types of entries (int, float, string, etc.)
# It then spits out a tuple of the different types and their relative frequencies in the column
def ratiosOfDifferentTypes(column):
    uniquerows = column.dropna().drop_duplicates()
    uniquerows = uniquerows.sample(n=min(1000,uniquerows.size))
    types = [type(entry) for entry in uniquerows]
    differenttypes = list(set(types))
    tally = [types.count(giventype) for giventype in differenttypes]
    total = sum(tally)*1.
    frequencies = [tallyelement / total for tallyelement in tally]
    return (differenttypes,frequencies)

# This function takes a column and decides which type its entries are meant to be like.
# It returns the type. If the entries are so mixed that it can't decide, it returns object.
def decideType(column):
    typesandratios = ratiosOfDifferentTypes(column)
    if max(typesandratios[1]) >= 0.8:
        # all the rows should probably be of the same type and some have been inputted incorrectly
        correcttype = typesandratios[0][typesandratios[1].index(max(typesandratios[1]))]
    else:
        # the rows have a very mixed type and it's not very clear what the correct type is
        correcttype = object
    return correcttype

# This function goes through all columns in the dataframe and returns the name of the columns that are dirty,
# i.e. that have mixed types of entries.
def findMixedTypes(dataframe):
    return [col for col in dataframe if len(ratiosOfDifferentTypes(dataframe[col])[1])>1]

# This function takes a dataframe and for each column says whether it's clean or dirty. If it's dirty,
# it tries to decide which type it should be.
def analyzeColumnTypes(dataframe):
    mixedtypecolumns = findMixedTypes(dataframe)
    if mixedtypecolumns==[]:
        print "All columns have a single type; they are 'clean'. (They may be incorrect though, or have NaNs)."
    else:
        print "The columns",mixedtypecolumns,"have mixed types:\n"
        correcttypes = [(colname,decideType(dataframe[colname])) for colname in mixedtypecolumns]
        for typ in correcttypes:
            if typ[1]==object:
                print " - \'" + typ[0] + "\'" + " is so mixed it's hard to tell the right type"
            else:
                print " - \'" + typ[0] + "\'" + " should be " + "\'" + typ[1].__name__ + "\'"
        print "\nAll other columns have a single type; they are 'clean'. (They may be incorrect though, or have NaNs)."
    return mixedtypecolumns

def findCleanStringTypes(dataframe):
    return [col for col in dataframe if ratiosOfDifferentTypes(dataframe[col])[0]==[str]]

def outlineNaNs(dataframe):
    totalnumberofNaNs = pd.isnull(dataframe).sum()
    percentageofNaNs = totalnumberofNaNs[totalnumberofNaNs > 0].astype(np.float64) / dataframe.shape[0]
    if len(percentageofNaNs)>0:
        print "Here are the columns with NaNs:"
        for kk in range(len(percentageofNaNs)):
            print " - \'" + percentageofNaNs.index[kk] + "\' has percentage of NaNs: \t" + str(percentageofNaNs[kk]) + " %"
        print "\nNo other columns have NaNs."
    else:
        print "No columns have NaNs."

def stripStartEndSpaces(listofdataframes):
    outputdataframes = listofdataframes
    for ii in range(len(listofdataframes)):
        cleanstringcolumns = findCleanStringTypes(listofdataframes[ii])
        for colname in cleanstringcolumns:
            outputdataframes[ii].loc[:,colname] = listofdataframes[ii][colname].str.strip()
    return outputdataframes

## Analyzing data types

Now we're going to print out which columns have dirty entries, i.e. have mixed types, and we'll try and guess what those entries should be

In [ ]:
mixedtypesindataframes = range(len(alldataframes))
for ii in range(len(alldataframes)):
    print "ANALYZING DATAFRAME FROM " + datafilenames[ii] + ":"
    print "===================================================="
    mixedtypesindataframes[ii] =  analyzeColumnTypes(alldataframes[ii])
    print "-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -"
    outlineNaNs(alldataframes[ii])
    print "===================================================="

## Finding strange / outlier data
Let's look at the UNIQUE values. We'll go through the dirty entries, as well as the purely-string entries.

In [ ]:
for ii in range(len(alldataframes)):
    print "ANALYZING DATAFRAME FROM " + datafilenames[ii] + ":"
    print "----------------------------------------------------"
    cleanstringcolumns = findCleanStringTypes(alldataframes[ii])
    for col in mixedtypesindataframes[ii] + cleanstringcolumns:
        print "Column '" + col + "' has the following unique entries:\n"
        print np.sort(alldataframes[ii][col].unique())
        print

## Removing bad datapoints

In [ ]:
#================================================================================================
# USER INPUT!
# Now we need to remove those rows of data that are missing critical information, i.e. remove 
# those rows that have a NaN for something very important.
criticalcolumns = [
    ["Survived","Pclass"],
    []
]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

for ii in range(len(alldataframes)):
    alldataframes[ii] = alldataframes[ii].dropna(subset = criticalcolumns[ii])

## Fill in missing data

In [ ]:
#================================================================================================
# USER INPUT!
# In the remaining columns there may be some NaNs, which should be replaced with some appropriate value.
# The variable whattodowithnans has the structure of a dictionary for every dataframe, i.e. [{"colname": valueforNaN,...},...]

whattodowithnans = [
    {"Survived": -1, "Pclass": -1,"Name": "Unknown name", "Sex": "Unspecified", 
     "Age": alldataframes[0]["Age"].dropna().mean(), "Ticket": "XXXXXX", 
     "Cabin": "XXX", "Embarked": "X"},
    {"Pclass": -1,"Name": "Unknown name", "Sex": "Unspecified", 
     "Age": alldataframes[0]["Age"].dropna().mean(), "Ticket": "XXXXXX", 
     "Cabin": "XXX", "Embarked": "X"}
]

#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

for ii in range(len(alldataframes)):
    alldataframes[ii] = alldataframes[ii].fillna(whattodowithnans[ii])

alldataframes = stripStartEndSpaces(alldataframes)

In [ ]:
#================================================================================================
# USER INPUT!
# Sometimes we want to fill the NaNs differently depending on the values of the other columns. Here we may do this

# Here we replace the ticket fare depending on the Pclass. We take the average Fare for a given Pclass
conditioncolumn = "Pclass"
shouldequal = [1,2,3]
toreplacecolumn = "Fare"

averageanswers = alldataframes[0].groupby(conditioncolumn)[toreplacecolumn].mean()
ii=0
for val in shouldequal:
    colrowstoreplacenans = alldataframes[ii][toreplacecolumn][alldataframes[ii][conditioncolumn]==val]
    indices = colrowstoreplacenans.index
    alldataframes[ii].loc[indices,toreplacecolumn] = colrowstoreplacenans.fillna(averageanswers[val])
ii=1
for val in shouldequal:
    colrowstoreplacenans = alldataframes[ii][toreplacecolumn][alldataframes[ii][conditioncolumn]==val]
    indices = colrowstoreplacenans.index
    alldataframes[ii].loc[indices,toreplacecolumn] = colrowstoreplacenans.fillna(averageanswers[val])

## Clean up dirty data

In [ ]:
#================================================================================================
# USER INPUT!
# After having run the previous cells you know which columns are clean and dirty.
# Look at the unique values of the dirty ones, given above. Use this information to clean them up

# In each dataframe, there are certain dirty columns that should be numeric
columnsthatshouldbenumeric = [
    ["Survived","Pclass","Age","SibSp","Fare"],
    ["Pclass","Age","SibSp","Fare"]
]
# In each column there will be some conventions on how the bad things are written out.
# For each column that should be numeric, we specify a tuple with the info
# (decimaldelimiter (a string),thousanddelimeter (a string), listofstringstoremove (a list))
structureofeachcolumn = [
    [
        (".",",",["-"," ","%"]),
        (".",",",["-"," "]),
        (".",",",["-"," ","%"]),
        (".",",",["-"," ","%"]),
        (".",",",["-"," ","%"])
    ],
    [
        (".",",",["-"," "]),
        (".",",",["-"," ","%"]),
        (".",",",["-"," ","%"]),
        (".",",",["-"," ","%"])
    ]
]

columnsthatshouldbestrings = [
    ["Name","Sex","Ticket","Cabin","Embarked"],
    ["Name","Sex","Ticket","Cabin","Embarked"]
]

columnsthatshouldbedatetimes = [
    [],
    []
]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

# This function takes a column that should be numeric but is all dirty with badly made strings. It removes
# the thousand-delimiters, it replaces the decimaldelimiters with periods, and removes any user-chosen 
# additional set of characters
def turnToNumeric(column,decimaldelimiter=".",thousanddelimeter=",",listofstringstoremove=["-"," ","%"]):
    toremoveregex = str(listofstringstoremove + [thousanddelimeter]).rstrip("]'").lstrip("'[").replace("', '","|")
    numericcolumn = pd.to_numeric(column.astype(str).str.replace(toremoveregex,"").str.replace(decimaldelimiter,"."))
    return numericcolumn

# This function takes a dirty column that should all be strings and turns it into such
def turnToString(column):
    return column.astype(str)

def turnToDate(column):
    return pd.to_datetime(column,dayfirst=True)

for ii in range(len(alldataframes)):
    for (jj,coltofix) in enumerate(columnsthatshouldbestrings[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToString(alldataframes[ii][coltofix])
    
    for (jj,coltofix) in enumerate(columnsthatshouldbedatetimes[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToDate(alldataframes[ii][coltofix])
        
    for (jj,coltofix) in enumerate(columnsthatshouldbenumeric[ii]):
        alldataframes[ii].loc[:,coltofix] = turnToNumeric(alldataframes[ii][coltofix],
                                                    structureofeachcolumn[ii][jj][0],
                                                    structureofeachcolumn[ii][jj][1],
                                                    structureofeachcolumn[ii][jj][2])

## Removing fake / duplicated data

In [ ]:
#================================================================================================
# USER INPUT!
# Some times there are duplicate entries. Some columns provide unique identifiers for the rows,
# for identifying unique entries (e.g. email address, full name, etc.). If there is no identifier
# column, we just plug the empty list [] into identifiercolumns.

identifiercolumns = [
    ["PassengerId"],
    ["PassengerId"]
]

#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

for ii in range(len(alldataframes)):
    if identifiercolumns[ii] != []:
        alldataframes[ii] = alldataframes[ii].drop_duplicates(identifiercolumns[ii])
    else:
        alldataframes[ii] = alldataframes[ii].drop_duplicates()

In [ ]:
#================================================================================================
# USER INPUT!
# Some data in our database can be fake data (e.g. data generating from testing whether the database works).
# This should be identified using .unique() on each column, above, and thrown away.
#  - It could be numeric data that is too big, too small, or at some impossible value.
#  - It could be impossible datetime stamps
#  - It could be strings that don't make sense; they might contain, end, or start with something bad.
#    They might be missing some parts in the string, or be too long or too short.

# NUMERIC DATA (these are the conditions the numbers should satisfy)
for ii in [0,1]:
    if ii==0:
        alldataframes[ii] = alldataframes[ii][(alldataframes[ii]["Survived"]==-1) | (alldataframes[ii]["Survived"]==0) | 
                                    (alldataframes[ii]["Survived"]==1)]
    alldataframes[ii] = alldataframes[ii][(alldataframes[ii]["Pclass"]==-1) | (alldataframes[ii]["Pclass"]==1) | 
                                    (alldataframes[ii]["Pclass"]==2) | (alldataframes[ii]["Pclass"]==3)]
    alldataframes[ii] = alldataframes[ii][(-0.1 < alldataframes[ii]["Age"]) & (alldataframes[ii]["Age"] < 120)]
    alldataframes[ii] = alldataframes[ii][(0.0 < alldataframes[ii]["Fare"])]

# TIMESTAMP DATA (these are the conditions the timestamps should satisfy)

# STRING DATA (these are the conditions the strings should satisfy)
for ii in [0,1]:
    alldataframes[ii] = alldataframes[ii][(alldataframes[ii]["Name"].str.contains("Pinko") & alldataframes[ii]["Name"].str.contains("Pallino"))==False]
    alldataframes[ii] = alldataframes[ii][(alldataframes[ii]["Ticket"]!="False") & (alldataframes[ii]["Ticket"]!="false")]
    alldataframes[ii] = alldataframes[ii][(alldataframes[ii]["Cabin"]!="Deck") & (alldataframes[ii]["Cabin"]!="deck")]

#================================================================================================

## Polishing up data

In [ ]:
#================================================================================================
# USER INPUT!
# Now the data is essentially clean. It is likely to still have outliers, and things that don't make sense.
# We'll polish it up even further.

alldataframes[0] = alldataframes[0].drop("PassengerId", 1)
alldataframes[1] = alldataframes[1].drop("PassengerId", 1)

# This turns a cabin number into simply the first letter of the cabin, i.e. the generic area on the ship

for ii in range(len(alldataframes)):
    alldataframes[ii].loc[:,"Cabin"] = alldataframes[ii]["Cabin"].apply(lambda x: x[0])

#================================================================================================

## Very basic exploratory analysis

In [ ]:
#================================================================================================
# Now we'll do some exploratory analysis. This will help find outliers and strange things in the data, 
# and in turn help us clean it further. Proper regressions etc. are in a separate notebook.

# READ THE OUTPUTS CAREFULLY!
# First we'll do a general "describe" on the data. This is useful to find outliers in min & max, etc.
alldataframes[0].describe()

In [ ]:
# Now we'll do a histogram of each column, to get an idea of the distribution of its values
alldataframes[0]._get_numeric_data().hist(bins=30, figsize=(8,8))
plt.show()

In [ ]:
alldataframes[0].groupby("Pclass").mean()

In [ ]:
alldataframes[0].groupby("Sex").mean()

In [ ]:
print alldataframes[0][alldataframes[0]["Age"] < 13]["Survived"].mean()
print alldataframes[0][alldataframes[0]["Age"] >= 13]["Survived"].mean()

## Save clean data in new files

In [ ]:
#================================================================================================
# USER INPUT!
# We are finished cleaning the data. We'll now output the clean data to a new csv file
# Here I specify the names of the clean-data files
outputfilenames = ["cleantrain.csv","cleantest.csv"]
#================================================================================================

# FROM HERE ON IT'S AUTOMATIC

for ii in range(len(alldataframes)):
    alldataframes[ii].to_csv(outputfilenames[ii])